# Demand Planning Tool - Production Forecast v3

This notebook creates a comprehensive demand planning analysis with:
- Historical sales analysis by Product Name/SKU and channel
- **Improved statistical forecasts** using 2-year product-type seasonality + weighted trend
- Seasonal indices calculated at **product type level** (pooled across all SKUs in a type)
- Recency-weighted trend per SKU (recent months weighted more heavily)
- Forecast pivot tabs show last 2 years of actuals alongside forecast for direct comparison
- Export to Google Sheets

## Forecasting Methodology (v3)
- **Seasonality**: Pooled across all SKUs within a product type, using the last 24 months of TOTAL channel data. Seasonal indices are stable because they draw on the full volume of a product type rather than a single item.
- **Trend**: Linear trend via weighted least-squares on each SKU's last 12 months (deseasonalized). Recent months weighted up to 12× heavier.
- **Base Level**: Exponentially weighted moving average of last 12 months per SKU.
- **Manual Growth Override**: Optional annual growth rate applied on top of data-driven trend.

## Setup Instructions
1. Upload your CSV file when prompted
2. Run all cells in order
3. Authenticate with Google when prompted
4. The output will be saved to your Google Drive

In [21]:
# Install required packages
!pip install gspread oauth2client pandas numpy openpyxl scipy -q

In [22]:
# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

from google.colab import files
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials

In [23]:
# Upload your CSV file
print("Please upload your sales data CSV file:")
uploaded = files.upload()
filename = list(uploaded.keys())[0]
print(f"\nFile '{filename}' uploaded successfully!")

Please upload your sales data CSV file:


Saving _SELECT_mts_created_date_mts_products__master__id_mts_products___202602131549.csv to _SELECT_mts_created_date_mts_products__master__id_mts_products___202602131549 (1).csv

File '_SELECT_mts_created_date_mts_products__master__id_mts_products___202602131549 (1).csv' uploaded successfully!


In [24]:
# Load and explore the data
df = pd.read_csv(filename)

print("Data Overview:")
print(f"Total rows: {len(df):,}")
print(f"Date range: {df['created_date'].min()} to {df['created_date'].max()}")
print(f"\nChannels: {df['orders__source'].unique()}")
print(f"Unique SKUs: {df['products__variants__sku'].nunique()}")
print(f"\nFirst few rows:")
df.head()

Data Overview:
Total rows: 83,658
Date range: 2022-01-01 to 2026-01-30

Channels: ['Direct-to-Consumer' 'Wholesale' 'Kristina Holey']
Unique SKUs: 275

First few rows:


,created_date,products__master__id,products__variants__sku,products__variants__title,products__root_product__title,products__product_type,products__vendor,orders__source,quantity,price,total_gross_sales,total_net_sales,total_sales
0,2022-01-01,19215247238,FG-10047,Vitamins C + E + Ferulic Serum - Retail (1 oz),Vitamins C + E + Ferulic Serum,SERUM,Marie Veronique,Direct-to-Consumer,7,90.0,630.0,630.0,630.0
1,2022-01-01,19215265286,FG-10029,Protective Day Oil - Retail (1 oz),Protective Day Oil,OIL,Marie Veronique,Direct-to-Consumer,7,65.0,455.0,447.2,447.2
2,2022-01-01,29370167116,FG-10006,Barrier Restore Serum - Retail (1 oz),Barrier Restore Serum,SERUM,Marie Veronique,Direct-to-Consumer,4,110.0,440.0,440.0,440.0
3,2022-01-01,32378481934372,FG-10005,Barrier Lipid Complex - Retail (1 oz),Barrier Lipid Complex,OIL,Marie Veronique,Direct-to-Consumer,4,95.0,380.0,380.0,380.0
4,2022-01-01,29370177804,FG-10038,Soothing B3 Serum - Retail (1 oz),Soothing B3 Serum,SERUM,Marie Veronique,Direct-to-Consumer,3,90.0,270.0,270.0,270.0


In [25]:
# Prepare data for analysis
df['created_date'] = pd.to_datetime(df['created_date'])
df['year_month'] = df['created_date'].dt.to_period('M')
df['year'] = df['created_date'].dt.year
df['month'] = df['created_date'].dt.month

# Aggregate to monthly level by SKU and channel
monthly_data = df.groupby(['year_month', 'products__variants__sku', 'orders__source'])['quantity'].sum().reset_index()
monthly_data['year_month_str'] = monthly_data['year_month'].astype(str)

# Get SKU details
sku_details = df.groupby('products__variants__sku').agg({
    'products__variants__title': 'first',
    'products__root_product__title': 'first',
    'products__product_type': 'first'
}).reset_index()

# Standardize product_name: prefer root product title, fall back to variant title
sku_details['product_name'] = sku_details['products__root_product__title'].fillna(
    sku_details['products__variants__title']
)

print("Data aggregated to monthly level")
print(f"Monthly records: {len(monthly_data):,}")
print(f"\nSample SKU → Product Name mapping:")
print(sku_details[['products__variants__sku', 'product_name', 'products__product_type']].head(10).to_string(index=False))

Data aggregated to monthly level
Monthly records: 7,546

Sample SKU → Product Name mapping:
products__variants__sku                            product_name products__product_type
        $20 Reward Code                         $20 Reward Code                   None
     000000000300088687 Quinton Hypertonic Ampoules 30 Servings                   None
           500-V0-40-DR                      Protective Day Oil                    OIL
           BLANKET-V4-1       HigherDose Blanket with No Insert                   WRAP
      BLANKET-W-1INSERT      HigherDose Blanket with One Insert                   WRAP
      BLANKET-W-3INSERT    Infrared Sauna Blanket by HigherDOSE                   WRAP
      BOTTLES+ CLOSURES                       BOTTLES+ CLOSURES                   None
                 CC3302                     The Cleansing Coins                   None
  DHL EXPRESS WORLDWIDE                   DHL EXPRESS WORLDWIDE                   None
              FG-100004               

In [26]:
# ============================================================
# STEP 1: BUILD PRODUCT-TYPE SEASONAL INDICES
# ============================================================
# Seasonality is pooled across ALL SKUs within each product type,
# using the last 24 months of TOTAL channel data.
# This gives stable, noise-resistant seasonal patterns.
# ============================================================

MONTH_NAMES = {
    1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr',
    5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug',
    9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
}

def build_product_type_seasonal_indices(monthly_data, sku_details, cutoff='2026-01'):
    """
    Calculate seasonal indices per product type per calendar month.
    Uses TOTAL channel (all channels combined) for the last 24 months up to cutoff.
    Returns a dict: {product_type: {month_number: index}}
    """
    # Work from TOTAL channel (sum all channels per SKU-month)
    total_monthly = monthly_data.groupby(['year_month_str', 'products__variants__sku'])['quantity'].sum().reset_index()

    # Attach product type
    total_monthly = total_monthly.merge(
        sku_details[['products__variants__sku', 'products__product_type']],
        on='products__variants__sku', how='left'
    )

    # Filter to last 24 months up to cutoff
    cutoff_period = pd.Period(cutoff, freq='M')
    start_period = cutoff_period - 23  # 24 months window
    total_monthly = total_monthly[
        (total_monthly['year_month_str'] >= str(start_period)) &
        (total_monthly['year_month_str'] <= cutoff)
    ].copy()

    total_monthly['cal_month'] = total_monthly['year_month_str'].str[5:7].astype(int)

    # Sum quantity by product_type + calendar month (across all SKUs and both years)
    pt_monthly = total_monthly.groupby(['products__product_type', 'cal_month'])['quantity'].sum().reset_index()

    # Calculate average monthly quantity per product type per calendar month
    # (divide by number of years represented — up to 2)
    # Then normalize so indices average to 1.0 across the 12 months
    seasonal_indices = {}
    product_types = pt_monthly['products__product_type'].unique()

    for pt in product_types:
        pt_data = pt_monthly[pt_monthly['products__product_type'] == pt].copy()

        # Build month → avg quantity mapping
        month_qty = {}
        for _, row in pt_data.iterrows():
            month_qty[int(row['cal_month'])] = row['quantity']

        # Fill missing months with average of available months
        if len(month_qty) > 0:
            avg_qty = np.mean(list(month_qty.values()))
        else:
            avg_qty = 1.0

        for m in range(1, 13):
            if m not in month_qty:
                month_qty[m] = avg_qty

        # Normalize: each index = month_qty / (sum of all months / 12)
        grand_avg = np.mean([month_qty[m] for m in range(1, 13)])
        if grand_avg > 0:
            indices = {m: month_qty[m] / grand_avg for m in range(1, 13)}
        else:
            indices = {m: 1.0 for m in range(1, 13)}

        seasonal_indices[pt] = indices

    return seasonal_indices


# Build product-type seasonal indices
pt_seasonal_indices = build_product_type_seasonal_indices(monthly_data, sku_details)

print(f"✅ Seasonal indices built for {len(pt_seasonal_indices)} product types")
print("\nProduct types found:")
for pt in sorted(pt_seasonal_indices.keys()):
    print(f"  • {pt}")

✅ Seasonal indices built for 25 product types

Product types found:
  • BACKBAR
  • BODY
  • BOOK
  • BUNDLE
  • CLEANSER
  • CONDITIONER
  • DUO
  • FREEGIFT_HIDDEN
  • GUA SHA
  • KIT
  • MASK
  • MIST
  • OIL
  • PACKAGING
  • PROTECT
  • SAMPLE
  • SERUM
  • SET
  • SHAMPOO
  • SOAP & LOTION DISPENSERS
  • SUNSCREEN
  • SUPPLEMENT
  • TINCTURE
  • VIRTUAL CONSULTATION
  • WRAP


In [27]:
# ============================================================
# STEP 2: DISPLAY SEASONALITY TABLE (Product Type × Month)
# ============================================================
# This shows the seasonal index for each product type by month.
# Index > 1.0 = that month is stronger than average
# Index < 1.0 = that month is weaker than average
# ============================================================

seasonality_rows = []
for pt, indices in sorted(pt_seasonal_indices.items()):
    row = {'Product Type': pt}
    for m in range(1, 13):
        row[MONTH_NAMES[m]] = round(indices[m], 3)
    seasonality_rows.append(row)

seasonality_display_df = pd.DataFrame(seasonality_rows)

print("📊 SEASONAL INDICES BY PRODUCT TYPE AND MONTH")
print("   (Based on last 24 months of TOTAL channel data)")
print("   Index > 1.0 = stronger than annual average | Index < 1.0 = weaker than annual average")
print()
print(seasonality_display_df.to_string(index=False))
print()
print("Note: Indices within each product type sum to 12.0 (average = 1.0)")

📊 SEASONAL INDICES BY PRODUCT TYPE AND MONTH
   (Based on last 24 months of TOTAL channel data)
   Index > 1.0 = stronger than annual average | Index < 1.0 = weaker than annual average

            Product Type   Jan   Feb   Mar   Apr   May   Jun   Jul   Aug   Sep   Oct   Nov   Dec
                 BACKBAR 0.842 1.000 1.000 1.000 1.000 1.474 1.684 0.947 1.158 0.526 0.737 0.632
                    BODY 0.812 0.379 0.612 0.719 0.512 0.590 0.569 1.085 0.647 2.998 1.580 1.496
                    BOOK 0.589 0.547 1.221 0.926 0.968 1.221 1.011 0.547 0.758 0.800 1.474 1.937
                  BUNDLE 1.537 0.743 0.824 0.566 0.355 0.283 0.313 0.460 0.667 0.372 4.921 0.959
                CLEANSER 0.949 1.052 1.038 0.790 0.881 0.880 1.028 0.922 1.215 0.807 1.562 0.877
             CONDITIONER 1.000 1.000 1.103 0.828 1.655 0.276 0.828 1.103 1.379 0.828 1.000 1.000
                     DUO 1.000 0.996 1.371 0.972 1.021 1.156 0.947 1.175 1.027 0.935 0.400 1.000
         FREEGIFT_HIDDEN 0.008 0.280 0

In [28]:
# ============================================================
# STEP 3: FORECASTING ENGINE (using product-type seasonality)
# ============================================================

def calculate_weighted_trend(deseasonalized_values):
    """
    Estimate trend (units/month) using weighted least-squares.
    Weights increase linearly so most recent = window weight, oldest = 1.
    """
    n = len(deseasonalized_values)
    if n < 4:
        return 0.0

    weights = np.arange(1, n + 1, dtype=float)
    x = np.arange(n, dtype=float)
    w = weights
    wx = (w * x).sum()
    wy = (w * deseasonalized_values).sum()
    wxx = (w * x * x).sum()
    wxy = (w * x * deseasonalized_values).sum()
    wsum = w.sum()

    denom = wsum * wxx - wx * wx
    if abs(denom) < 1e-10:
        return 0.0

    slope = (wsum * wxy - wx * wy) / denom
    return slope


def exponential_weighted_mean(arr, alpha=0.15):
    """
    Exponentially weighted mean — most recent observation has highest weight.
    """
    n = len(arr)
    weights = np.array([(1 - alpha) ** (n - 1 - i) for i in range(n)])
    return np.dot(weights, arr) / weights.sum()


def calculate_forecast(historical_values, calendar_months, product_type,
                       pt_seasonal_indices, forecast_calendar_months,
                       growth_rate=0.0):
    """
    Forecast using PRODUCT TYPE seasonal indices.

    Parameters:
    - historical_values       : numpy array of quantities (sorted oldest→newest)
    - calendar_months         : numpy array of calendar month numbers (1-12) for history
    - product_type            : string product type for seasonal index lookup
    - pt_seasonal_indices     : dict {product_type: {month: index}}
    - forecast_calendar_months: list of calendar month numbers (1-12) for forecast
    - growth_rate             : manual annual growth override
    """
    n = len(historical_values)
    forecast_periods = len(forecast_calendar_months)

    # Get seasonal indices for this product type (fallback to flat 1.0)
    seasonal_indices = pt_seasonal_indices.get(product_type, {m: 1.0 for m in range(1, 13)})

    if n < 4:
        avg = np.mean(historical_values) if n > 0 else 0
        forecasts = []
        for cal_m in forecast_calendar_months:
            forecasts.append(int(round(max(0, avg * seasonal_indices.get(cal_m, 1.0)))))
        return forecasts

    # Deseasonalize last 12 months of history
    window = min(n, 12)
    recent_vals = historical_values[-window:]
    recent_months = calendar_months[-window:]

    deseason_recent = np.array([
        v / seasonal_indices.get(int(m), 1.0) if seasonal_indices.get(int(m), 1.0) > 0 else v
        for v, m in zip(recent_vals, recent_months)
    ])

    # Base level: exponentially weighted mean of deseasonalized recent
    base_level = exponential_weighted_mean(deseason_recent, alpha=0.15)

    # Trend: weighted least squares on deseasonalized recent
    trend_slope = calculate_weighted_trend(deseason_recent)

    # Cap trend at ±2.5% of base per month
    max_slope = base_level * 0.025 if base_level > 0 else 1.0
    trend_slope = np.clip(trend_slope, -max_slope, max_slope)

    # Manual growth → monthly multiplier
    monthly_growth = (1 + growth_rate) ** (1 / 12) - 1

    forecasts = []
    for i, cal_month in enumerate(forecast_calendar_months):
        projected_base = base_level + trend_slope * (i + 1)
        growth_factor = (1 + monthly_growth) ** (i + 1)
        projected_base *= growth_factor
        seasonal_factor = seasonal_indices.get(int(cal_month), 1.0)
        forecast = projected_base * seasonal_factor
        forecasts.append(int(round(max(0, forecast))))

    return forecasts


print("✅ Forecasting engine loaded (product-type seasonality)")

✅ Forecasting engine loaded (product-type seasonality)


In [29]:
# ============================================================================
# TUNE YOUR FORECAST GROWTH RATE HERE
# ============================================================================
# Manual override added on top of the data-driven trend.
# Leave at 0.0 to rely entirely on the statistical trend from your data.
#   0.10  = add 10% growth on top of data trend
#   0.0   = pure statistical forecast (recommended starting point)
#  -0.05  = force 5% decline on top of data trend

FORECAST_GROWTH_RATE = 0.0

print(f"Manual Growth Rate Override: {FORECAST_GROWTH_RATE*100:.1f}% annually")

Manual Growth Rate Override: 0.0% annually


In [30]:
# ============================================================
# STEP 4: GENERATE FORECASTS
# ============================================================
# INACTIVITY RULE: If a SKU had zero sales in the last 6 months
# (across ALL channels combined), it is treated as inactive and
# skipped — no forecast is generated for it in any channel.
# Inactive SKUs still appear in historical tabs but show 0 forecast.
# ============================================================

forecast_months = pd.period_range('2026-02', '2026-12', freq='M')
forecast_cal_months = [p.month for p in forecast_months]

all_channels = ['Direct-to-Consumer', 'Wholesale', 'TOTAL']

# --- Build inactive SKU list ---
# Use the 6 months ending at the last history month (2026-01)
HISTORY_CUTOFF = '2026-01'
cutoff_period = pd.Period(HISTORY_CUTOFF, freq='M')
lookback_start = str(cutoff_period - 5)  # 6-month window inclusive

# Sum TOTAL sales per SKU over the last 6 months (all channels)
recent_sales = monthly_data[
    (monthly_data['year_month_str'] >= lookback_start) &
    (monthly_data['year_month_str'] <= HISTORY_CUTOFF)
].groupby('products__variants__sku')['quantity'].sum()

# A SKU is inactive if it has NO sales at all in those 6 months
inactive_skus = set(recent_sales[recent_sales == 0].index)

# Also flag SKUs that don't appear at all in the recent window
all_known_skus = set(monthly_data['products__variants__sku'].unique())
skus_with_any_recent = set(recent_sales.index)
inactive_skus |= (all_known_skus - skus_with_any_recent)

print(f"Inactivity window: {lookback_start} to {HISTORY_CUTOFF} (6 months)")
print(f"Total SKUs: {len(all_known_skus)}")
print(f"Inactive SKUs (no sales in last 6 months): {len(inactive_skus)}")
print(f"Active SKUs to forecast: {len(all_known_skus) - len(inactive_skus)}")
if inactive_skus:
    inactive_names = sku_details[
        sku_details['products__variants__sku'].isin(inactive_skus)
    ][['products__variants__sku', 'product_name']].values.tolist()
    print(f"\nInactive SKUs skipped:")
    for sku_id, name in inactive_names:
        print(f"  • {name} ({sku_id})")

# --- Generate forecasts (active SKUs only) ---
forecast_results = []
skipped_counts = {ch: 0 for ch in all_channels}

for channel in all_channels:
    print(f"\nGenerating forecasts for {channel}...")

    if channel == 'TOTAL':
        channel_data = monthly_data.groupby(['year_month_str', 'products__variants__sku'])['quantity'].sum().reset_index()
    else:
        channel_data = monthly_data[monthly_data['orders__source'] == channel].copy()

    channel_skus = channel_data['products__variants__sku'].unique()

    for sku in channel_skus:
        sku_match = sku_details[sku_details['products__variants__sku'] == sku]
        if len(sku_match) == 0:
            continue
        sku_info = sku_match.iloc[0]
        product_type = sku_info['products__product_type']

        # INACTIVITY CHECK — skip if no sales in last 6 months
        if sku in inactive_skus:
            skipped_counts[channel] += 1
            continue

        sku_data = channel_data[channel_data['products__variants__sku'] == sku]

        # Historical data up to cutoff, sorted chronologically
        historical = sku_data[sku_data['year_month_str'] <= HISTORY_CUTOFF].sort_values('year_month_str')
        hist_values = historical['quantity'].values.astype(float)
        hist_cal_months = np.array([int(ym[5:7]) for ym in historical['year_month_str'].values])

        # Forecast using product-type seasonality
        forecasts = calculate_forecast(
            hist_values,
            hist_cal_months,
            product_type,
            pt_seasonal_indices,
            forecast_cal_months,
            FORECAST_GROWTH_RATE
        )

        for month, forecast_qty in zip(forecast_months, forecasts):
            forecast_results.append({
                'channel': channel,
                'product_name': sku_info['product_name'],
                'sku': sku,
                'product_type': product_type,
                'month': str(month),
                'forecast_qty': forecast_qty
            })

forecast_df = pd.DataFrame(forecast_results)
print(f"\n✅ Forecasts generated: {len(forecast_df):,} records")
print(f"   SKUs skipped per channel: { {k: v for k, v in skipped_counts.items()} }")

Inactivity window: 2025-08 to 2026-01 (6 months)
Total SKUs: 275
Inactive SKUs (no sales in last 6 months): 150
Active SKUs to forecast: 125

Inactive SKUs skipped:
  • Protective Day Oil (500-V0-40-DR)
  • HigherDose Blanket with No Insert (BLANKET-V4-1)
  • Infrared Sauna Blanket by HigherDOSE (BLANKET-W-3INSERT)
  • BOTTLES+ CLOSURES (BOTTLES+ CLOSURES)
  • DHL EXPRESS WORLDWIDE (DHL EXPRESS WORLDWIDE)
  • Aftershave Tonic (FG-10002)
  • Replenishing Oil Cleanser (FG-100036)
  • Treatment Mist (FG-100043)
  • Daily Exfoliating Cleanser (FG-10011)
  • Eye Repair Serum (FG-10015)
  • Mildly Foaming Shaving Oil (FG-10024)
  • Multi-Retinol Night Emulsion (FG-10025E)
  • Redness Relief Serum (FG-10033)
  • Rejuvenating Night Oil - Vegan (FG-10035)
  • Shave Prep + Daily Wash (FG-10037)
  • Treatment Cleanser (FG-10040)
  • Treatment Mist (FG-10043)
  • Treatment Oil (FG-10044)
  • Treatment Retinol Serum (FG-10045)
  • Treatment Serum (FG-10046)
  • Body Barrier Emulsion (FG-10067)
  • 

In [31]:
# ============================================================
# STEP 5: BUILD FORECAST COMPARISON PIVOT
# (2024 actuals | 2025 actuals | 2026 forecast — all months)
# ============================================================

def build_comparison_pivot(channel, monthly_data, forecast_df, sku_details):
    """
    Build a pivot table showing:
      Rows: Product Name | SKU
      Columns: 2024-01 ... 2024-12, 2025-01 ... 2025-12, 2026-01 (actual), 2026-02 ... 2026-12 (forecast)
    Adds a row-level label prefix so actuals vs forecast are clear.
    """
    # --- Historical actuals ---
    if channel == 'TOTAL':
        hist_data = monthly_data.groupby(['year_month_str', 'products__variants__sku'])['quantity'].sum().reset_index()
    else:
        hist_data = monthly_data[monthly_data['orders__source'] == channel].copy()
        hist_data = hist_data[['year_month_str', 'products__variants__sku', 'quantity']]

    # Only keep last 2 years of actuals (2024 + 2025) plus 2026-01
    hist_data = hist_data[
        (hist_data['year_month_str'] >= '2024-01') &
        (hist_data['year_month_str'] <= '2026-01')
    ].copy()

    hist_data = hist_data.merge(
        sku_details[['products__variants__sku', 'product_name']],
        on='products__variants__sku', how='left'
    )
    hist_data['type_flag'] = 'Actual'

    # --- Forecast (2026-02 to 2026-12) ---
    fcst_data = forecast_df[forecast_df['channel'] == channel][['month', 'sku', 'product_name', 'forecast_qty']].copy()
    fcst_data.columns = ['year_month_str', 'products__variants__sku', 'product_name', 'quantity']
    fcst_data['type_flag'] = 'Forecast'

    # Combine
    combined = pd.concat([hist_data, fcst_data], ignore_index=True)
    # Pivot with two index levels: product_name and sku
    combined['product_name'] = combined['product_name'].fillna('')
    combined['products__variants__sku'] = combined['products__variants__sku'].fillna('')

    pivot = combined.pivot_table(
        index=['product_name', 'products__variants__sku'],
        columns='year_month_str',
        values='quantity',
        fill_value=0,
        aggfunc='sum'
    )

    # Sort columns chronologically
    pivot = pivot.reindex(sorted(pivot.columns), axis=1)

    return pivot


comparison_pivots = {}
for channel in all_channels:
    comparison_pivots[channel] = build_comparison_pivot(channel, monthly_data, forecast_df, sku_details)

print("✅ Comparison pivots built (2024 actuals | 2025 actuals | 2026 forecast)")
# Show column range for verification
sample_cols = list(comparison_pivots['TOTAL'].columns)
print(f"Columns: {sample_cols[0]} → {sample_cols[-1]} ({len(sample_cols)} months)")

✅ Comparison pivots built (2024 actuals | 2025 actuals | 2026 forecast)
Columns: 2024-01 → 2026-12 (36 months)


In [32]:
# Create comparison DataFrames and helper structures
comparison_results = []
for channel in all_channels:
    channel_forecasts = forecast_df[forecast_df['channel'] == channel]
    total_channel_forecast = channel_forecasts['forecast_qty'].sum()
    product_type_forecasts = channel_forecasts.groupby('product_type')['forecast_qty'].sum().reset_index()
    total_by_product_type = product_type_forecasts['forecast_qty'].sum()
    total_by_sku = channel_forecasts.groupby('sku')['forecast_qty'].sum().sum()

    comparison_results.append({
        'Channel': channel,
        'A - Total Forecast': round(total_channel_forecast, 0),
        'B - Product Type': round(total_by_product_type, 0),
        'C - Item Level': round(total_by_sku, 0),
        'B vs A Diff': round(total_by_product_type - total_channel_forecast, 2),
        'C vs A Diff': round(total_by_sku - total_channel_forecast, 2)
    })

comparison_df = pd.DataFrame(comparison_results)

# Product type breakdown
product_type_details = []
for channel in all_channels:
    channel_forecasts = forecast_df[forecast_df['channel'] == channel]
    pt_summary = channel_forecasts.groupby('product_type').agg(
        Total_Forecast_Qty=('forecast_qty', 'sum'),
        Num_SKUs=('sku', 'nunique')
    ).reset_index()
    pt_summary.columns = ['Product_Type', 'Total_Forecast_Qty', 'Num_SKUs']
    pt_summary['Channel'] = channel
    pt_summary['Avg_Per_SKU'] = (pt_summary['Total_Forecast_Qty'] / pt_summary['Num_SKUs']).round(1)
    channel_total = pt_summary['Total_Forecast_Qty'].sum()
    pt_summary['Pct_of_Channel'] = ((pt_summary['Total_Forecast_Qty'] / channel_total) * 100).round(1)
    product_type_details.append(pt_summary)

product_type_df = pd.concat(product_type_details, ignore_index=True)
product_type_df = product_type_df[['Channel', 'Product_Type', 'Num_SKUs', 'Total_Forecast_Qty', 'Avg_Per_SKU', 'Pct_of_Channel']]

# Channel summary
summary_by_channel = []
for channel in all_channels:
    if channel == 'TOTAL':
        channel_monthly = monthly_data.groupby(['year_month_str'])['quantity'].sum().reset_index()
    else:
        channel_monthly = monthly_data[monthly_data['orders__source'] == channel].groupby(['year_month_str'])['quantity'].sum().reset_index()
    channel_monthly['year'] = channel_monthly['year_month_str'].str[:4]
    yearly = channel_monthly.groupby('year')['quantity'].sum()
    forecast_total = forecast_df[forecast_df['channel'] == channel]['forecast_qty'].sum()
    summary_by_channel.append({
        'Channel': channel,
        '2022_Total': int(yearly.get('2022', 0)),
        '2023_Total': int(yearly.get('2023', 0)),
        '2024_Total': int(yearly.get('2024', 0)),
        '2025_Total': int(yearly.get('2025', 0)),
        '2026_YTD': int(yearly.get('2026', 0)),
        '2026_Forecast': int(forecast_total)
    })

summary_df = pd.DataFrame(summary_by_channel)
summary_df['YoY_Growth'] = ((summary_df['2026_Forecast'] - summary_df['2025_Total']) / summary_df['2025_Total'] * 100).round(1)

print("✅ Summary dataframes ready")

✅ Summary dataframes ready


In [33]:
# ============================================================
# STEP 6: BUILD AGGREGATED BY YEAR DATA
# Includes 2026 Full Year Total Forecast
# ============================================================

trend_data = []
total_monthly_all = monthly_data.groupby(['year_month_str', 'products__variants__sku'])['quantity'].sum().reset_index()

for sku in sku_details['products__variants__sku'].unique():
    sku_info = sku_details[sku_details['products__variants__sku'] == sku].iloc[0]

    sku_data = total_monthly_all[total_monthly_all['products__variants__sku'] == sku].copy()
    sku_data['year'] = sku_data['year_month_str'].str[:4]
    yearly = sku_data.groupby('year')['quantity'].sum()

    # 2026 actuals (Jan only so far)
    ytd_actual = int(yearly.get('2026', 0))

    # 2026 forecast (Feb-Dec)
    sku_fcst_2026 = forecast_df[
        (forecast_df['sku'] == sku) & (forecast_df['channel'] == 'TOTAL')
    ]['forecast_qty'].sum()

    full_year_est = ytd_actual + sku_fcst_2026

    trend_data.append({
        'Product Name': sku_info['product_name'],
        'SKU': sku,
        'Product_Type': sku_info['products__product_type'],
        '2022': int(yearly.get('2022', 0)),
        '2023': int(yearly.get('2023', 0)),
        '2024': int(yearly.get('2024', 0)),
        '2025': int(yearly.get('2025', 0)),
        '2026_Jan_Actual': ytd_actual,
        '2026_Feb_Dec_Forecast': int(round(sku_fcst_2026)),
        '2026_Total_Forecast': int(round(full_year_est))
    })

trend_df = pd.DataFrame(trend_data)

# Add a TOTALS row at the bottom
totals_row = {
    'Product Name': '** TOTAL **',
    'SKU': '',
    'Product_Type': '',
    '2022': trend_df['2022'].sum(),
    '2023': trend_df['2023'].sum(),
    '2024': trend_df['2024'].sum(),
    '2025': trend_df['2025'].sum(),
    '2026_Jan_Actual': trend_df['2026_Jan_Actual'].sum(),
    '2026_Feb_Dec_Forecast': int(round(trend_df['2026_Feb_Dec_Forecast'].sum())),
    '2026_Total_Forecast': int(round(trend_df['2026_Total_Forecast'].sum()))
}
trend_df = pd.concat([trend_df, pd.DataFrame([totals_row])], ignore_index=True)

print("✅ Aggregated By Year data ready")
print(f"\n2026 Total Forecast (all SKUs): {totals_row['2026_Total_Forecast']:,.0f} units")
print(f"  Jan Actual:       {totals_row['2026_Jan_Actual']:,.0f}")
print(f"  Feb-Dec Forecast: {totals_row['2026_Feb_Dec_Forecast']:,.0f}")

✅ Aggregated By Year data ready

2026 Total Forecast (all SKUs): 173,609 units
  Jan Actual:       10,141
  Feb-Dec Forecast: 163,468


In [34]:
# Authenticate with Google
import google.auth
auth.authenticate_user()
credentials, project = google.auth.default()
gc = gspread.authorize(credentials)
print("Authenticated with Google")

Authenticated with Google


In [35]:
# Create new Google Sheet
sheet_name = f"Demand_Planning_{datetime.now().strftime('%Y%m%d_%H%M')}"
sh = gc.create(sheet_name)
print(f"Created Google Sheet: {sheet_name}")
print(f"URL: https://docs.google.com/spreadsheets/d/{sh.id}")

Created Google Sheet: Demand_Planning_20260214_0002
URL: https://docs.google.com/spreadsheets/d/1KTV8BnBjkd7yPYFnZ4LIEjIOGFoFk3Rj44tWbul2kpg


In [36]:
# ============================================================
# WRITE: Summary Dashboard
# ============================================================
summary_sheet = sh.sheet1
summary_sheet.update_title('Summary Dashboard')

summary_sheet.update('A1', [['DEMAND PLANNING SUMMARY']])
summary_sheet.update('A3', [['Historical Period: 2022-01 through 2026-01']])
summary_sheet.update('A4', [['Forecast Period: 2026-02 through 2026-12']])

# --- Methodology explanation ---
method_rows = [
    ['FORECASTING METHODOLOGY'],
    [''],
    ['SEASONALITY (Product-Type Level)'],
    ['  Seasonal indices are calculated by pooling all SKUs within each product type together, using the last 24 months'],
    ['  of TOTAL channel sales data. For each calendar month (Jan–Dec), we compute the average volume relative to the'],
    ['  overall monthly average for that product type. This produces a stable index (e.g. 1.42 = 42% above average,'],
    ['  0.71 = 29% below average) that is shared by all SKUs in the same product type. Using product-type pooling'],
    ['  rather than per-item indices prevents noisy, low-volume SKUs from generating unreliable seasonal patterns.'],
    [''],
    ['TREND (Per-SKU, Recency-Weighted)'],
    ['  A linear trend is estimated for each SKU individually using the last 12 months of deseasonalized sales.'],
    ['  Weighted least-squares regression is used, where the most recent month carries 12x the weight of the'],
    ['  oldest month in the window. This means recent acceleration or deceleration in demand has a much stronger'],
    ['  influence on the slope than older data. The trend slope is capped at ±2.5% of the base level per month'],
    ['  (~30% annually) to prevent runaway projections on sparse or erratic SKUs.'],
    [''],
    ['BASE LEVEL (Per-SKU)'],
    ['  The deseasonalized base level is calculated as an exponentially weighted moving average of the last 12'],
    ['  months (decay factor alpha=0.15). Recent months carry significantly more weight than older months,'],
    ['  so the base level responds to recent demand shifts while remaining stable against one-off spikes.'],
    [''],
    ['FORECAST CALCULATION'],
    ['  For each future month: Forecast = (Base Level + Trend × Steps Ahead) × Seasonal Index × Growth Factor'],
    ['  Seasonal Index: product-type index for that calendar month'],
    ['  Growth Factor: optional manual override (default 0% = pure statistical forecast)'],
    ['  All forecasts are rounded to whole units — no fractional quantities.'],
    [''],
    ['INACTIVITY RULE'],
    ['  Any SKU with zero total sales across all channels in the 6-month window prior to the forecast start'],
    ['  is classified as inactive and receives no forecast. These SKUs still appear in historical views.'],
    [''],
    ['WHAT THIS FORECAST DOES NOT CAPTURE'],
    ['  The statistical model extrapolates demand patterns from historical sell-through data. It does not'],
    ['  account for the following factors, which should be applied as manual judgment on top of the forecast:'],
    ['  • Promotions & Discounts: Heavy discounting (e.g. sitewide sales) inflates historical volume in'],
    ['    those months. The model will partially absorb this into the base level and seasonal index,'],
    ['    which can cause future months to be over- or under-forecast relative to promo intent.'],
    ['  • New Product Launches: SKUs with < 6 months of history have limited trend signal. Review'],
    ['    their forecasts manually and consider applying a growth override.'],
    ['  • Planned Price Changes: A price increase typically suppresses volume; a reduction lifts it.'],
    ['    Neither is visible to the model.'],
    ['  • Inventory / Supply Constraints: Stockouts in the historical window appear as zero demand,'],
    ['    causing the model to underestimate true underlying demand for those periods.'],
    ['  • Channel Mix Shifts: If volume is intentionally being moved between DTC and Wholesale,'],
    ['    the channel-level forecasts will not reflect that intent.'],
    ['  • Discontinued SKUs: Captured by the inactivity rule but only if sales went to zero in the'],
    ['    last 6 months. SKUs being wound down gradually will still receive a (likely too-high) forecast.'],
    [''],
]
summary_sheet.update('A6', method_rows)

method_end_row = 6 + len(method_rows) + 1
summary_sheet.update(f'A{method_end_row}', [['CHANNEL SUMMARY']])
summary_data = [summary_df.columns.tolist()] + summary_df.values.tolist()
summary_sheet.update(f'A{method_end_row + 2}', summary_data)

comparison_start = method_end_row + 12
summary_sheet.update(f'A{comparison_start}', [['FORECAST AGGREGATION COMPARISON']])
comparison_data = [comparison_df.columns.tolist()] + comparison_df.values.tolist()
summary_sheet.update(f'A{comparison_start + 2}', comparison_data)

summary_sheet.format('A1', {'textFormat': {'bold': True, 'fontSize': 14}})
summary_sheet.format('A6', {'textFormat': {'bold': True, 'fontSize': 12}})
summary_sheet.format('A8', {'textFormat': {'bold': True, 'underline': True}})
summary_sheet.format('A14', {'textFormat': {'bold': True, 'underline': True}})
summary_sheet.format('A21', {'textFormat': {'bold': True, 'underline': True}})
summary_sheet.format('A27', {'textFormat': {'bold': True, 'underline': True}})
summary_sheet.format('A33', {'textFormat': {'bold': True, 'underline': True}})
summary_sheet.format(f'A{method_end_row}', {'textFormat': {'bold': True, 'fontSize': 12}})
summary_sheet.format(f'A{method_end_row + 2}:H{method_end_row + 2}', {
    'textFormat': {'bold': True, 'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}},
    'backgroundColor': {'red': 0.259, 'green': 0.522, 'blue': 0.957}
})
summary_sheet.format(f'A{comparison_start}', {'textFormat': {'bold': True, 'fontSize': 12}})
summary_sheet.format(f'A{comparison_start + 2}:F{comparison_start + 2}', {
    'textFormat': {'bold': True, 'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}},
    'backgroundColor': {'red': 0.259, 'green': 0.522, 'blue': 0.957}
})

print("Summary Dashboard created with full methodology explanation")

Summary Dashboard created with full methodology explanation


In [37]:
# ============================================================
# WRITE: Product Type Breakdown (with Seasonality Table)
# ============================================================
pt_sheet = sh.add_worksheet(title='Product Type Breakdown', rows=1000, cols=30)

# Section 1: Seasonality indices
pt_sheet.update('A1', [['SEASONAL INDICES BY PRODUCT TYPE AND MONTH']])
pt_sheet.update('A2', [['Based on last 24 months of TOTAL channel data. Index > 1.0 = stronger than average. Normalized so each row averages to 1.0.']])

seas_data = [seasonality_display_df.columns.tolist()] + seasonality_display_df.values.tolist()
pt_sheet.update('A4', seas_data)

# Header formatting for seasonality table
pt_sheet.format('A1', {'textFormat': {'bold': True, 'fontSize': 13}})
num_pt_rows = len(seasonality_display_df)
header_range = f'A4:M4'
pt_sheet.format(header_range, {
    'textFormat': {'bold': True, 'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}},
    'backgroundColor': {'red': 0.18, 'green': 0.53, 'blue': 0.33}  # green
})

# Section 2: Forecast summary by product type
sep_row = num_pt_rows + 7  # leave a gap
pt_sheet.update(f'A{sep_row}', [['FORECAST SUMMARY BY PRODUCT TYPE AND CHANNEL']])
pt_sheet.format(f'A{sep_row}', {'textFormat': {'bold': True, 'fontSize': 13}})

pt_data = [product_type_df.columns.tolist()] + product_type_df.values.tolist()
pt_sheet.update(f'A{sep_row + 2}', pt_data)
pt_sheet.format(f'A{sep_row + 2}:F{sep_row + 2}', {
    'textFormat': {'bold': True, 'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}},
    'backgroundColor': {'red': 0.259, 'green': 0.522, 'blue': 0.957}
})

print("Product Type Breakdown sheet created (with seasonality indices)")

Product Type Breakdown sheet created (with seasonality indices)


In [38]:
# ============================================================
# WRITE: Channel Forecast + Comparison Pivot sheets
# Each channel gets ONE sheet: 2024 actuals | 2025 actuals | 2026 (Jan actual + Feb-Dec forecast)
# ============================================================

FORECAST_START = '2026-02'  # first forecast month

for channel in all_channels:
    print(f"Creating forecast sheet for {channel}...")

    ws = sh.add_worksheet(title=f"{channel} - Forecast", rows=2000, cols=150)

    pivot = comparison_pivots[channel]
    all_cols = list(pivot.columns)  # chronologically sorted month strings

    # Identify which columns are forecast vs actual
    # 2026-01 = actual, 2026-02 onward = forecast
    actual_cols = [c for c in all_cols if c < FORECAST_START]
    forecast_cols = [c for c in all_cols if c >= FORECAST_START]

    # Build header rows
    # Row 1: title
    ws.update('A1', [[f'{channel} — Historical vs Forecast']])

    # Row 2: year group labels  (2024 actuals / 2025 actuals / 2026 actual+forecast)
    year_label_row = ['Product Name', 'SKU']
    prev_year = None
    col_labels = []
    for c in all_cols:
        yr = c[:4]
        is_fcst = (c >= FORECAST_START)
        label = f"{yr} {'[FORECAST]' if is_fcst else '[ACTUAL]'}"
        col_labels.append(label)
    year_label_row = ['Product Name', 'SKU'] + col_labels

    # Row 3: month column headers
    month_header_row = ['Product Name', 'SKU'] + all_cols

    # Data rows — MultiIndex: (product_name, sku)
    data_rows = []
    for idx in pivot.index:
        prod_name, sku_code = idx
        row = [prod_name, sku_code] + [int(round(pivot.loc[idx, c])) if c in pivot.columns else 0 for c in all_cols]
        data_rows.append(row)

    # Add a TOTALS row
    totals_row_data = ['** CHANNEL TOTAL **', '']
    for c in all_cols:
        if c in pivot.columns:
            totals_row_data.append(int(round(pivot[c].sum())))
        else:
            totals_row_data.append(0)
    data_rows.append(totals_row_data)

    ws.update('A2', [year_label_row])
    ws.update('A3', [month_header_row])
    ws.update('A4', data_rows)

    # Format headers
    ws.format('A1', {'textFormat': {'bold': True, 'fontSize': 13}})

    # Row 2: alternating year/type shading
    # Row 3: column month headers — bold
    ws.format('A3', {'textFormat': {'bold': True}})

    # Color the actual columns header (row 2) in grey-blue
    num_actual = len(actual_cols)
    num_forecast = len(forecast_cols)

    if num_actual > 0:
        # Columns B onward for actual (1-indexed → col B = 2)
        import string
        def col_letter(n):
            """Convert 1-based column index to letter(s)."""
            result = ''
            while n > 0:
                n, remainder = divmod(n - 1, 26)
                result = chr(65 + remainder) + result
            return result

        actual_start_col = col_letter(2)  # B
        actual_end_col = col_letter(1 + num_actual)
        ws.format(f'{actual_start_col}2:{actual_end_col}2', {
            'textFormat': {'bold': True, 'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}},
            'backgroundColor': {'red': 0.36, 'green': 0.44, 'blue': 0.56}  # slate
        })
        ws.format(f'{actual_start_col}3:{actual_end_col}3', {
            'textFormat': {'bold': True}
        })

    if num_forecast > 0:
        fcst_start_col = col_letter(2 + num_actual)
        fcst_end_col = col_letter(1 + num_actual + num_forecast)
        ws.format(f'{fcst_start_col}2:{fcst_end_col}2', {
            'textFormat': {'bold': True, 'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}},
            'backgroundColor': {'red': 0.18, 'green': 0.53, 'blue': 0.33}  # green for forecast
        })
        ws.format(f'{fcst_start_col}3:{fcst_end_col}3', {
            'textFormat': {'bold': True, 'foregroundColor': {'red': 0.1, 'green': 0.5, 'blue': 0.2}}
        })

    print(f"  → {len(all_cols)} months ({num_actual} actuals + {num_forecast} forecast)")

print("\nAll channel forecast sheets created")

Creating forecast sheet for Direct-to-Consumer...
  → 36 months (25 actuals + 11 forecast)
Creating forecast sheet for Wholesale...
  → 36 months (25 actuals + 11 forecast)
Creating forecast sheet for TOTAL...
  → 36 months (25 actuals + 11 forecast)

All channel forecast sheets created


In [39]:
# ============================================================
# WRITE: Aggregated By Year
# ============================================================
ws_trend = sh.add_worksheet(title="Aggregated By Year", rows=1000, cols=20)

ws_trend.update('A1', [['AGGREGATED BY YEAR — TOTAL CHANNEL (All SKUs)']])
ws_trend.update('A2', [['2026_Total_Forecast = Jan Actual + Feb–Dec Statistical Forecast']])

trend_sheet_data = [trend_df.columns.tolist()] + [
    [str(v) if not isinstance(v, (int, float)) else v for v in row]
    for row in trend_df.values.tolist()
]
ws_trend.update('A4', trend_sheet_data)

# Format header row
ws_trend.format('A1', {'textFormat': {'bold': True, 'fontSize': 13}})
ws_trend.format(f'A4:{chr(64 + len(trend_df.columns))}4', {
    'textFormat': {'bold': True, 'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}},
    'backgroundColor': {'red': 0.259, 'green': 0.522, 'blue': 0.957}
})

# Bold the TOTAL row at the bottom
total_row_idx = len(trend_df) + 4  # 1-indexed sheet row
ws_trend.format(f'A{total_row_idx}:{chr(64 + len(trend_df.columns))}{total_row_idx}', {
    'textFormat': {'bold': True},
    'backgroundColor': {'red': 0.95, 'green': 0.95, 'blue': 0.75}  # light yellow highlight
})

print("Aggregated By Year sheet created (with 2026 Total Forecast column)")

Aggregated By Year sheet created (with 2026 Total Forecast column)


In [40]:
# Final output
print("\n" + "="*80)
print("DEMAND PLANNING TOOL v3 CREATED SUCCESSFULLY!")
print("="*80)
print(f"\nGoogle Sheet Name: {sheet_name}")
print(f"URL: https://docs.google.com/spreadsheets/d/{sh.id}")
print(f"\nSheets created:")
for worksheet in sh.worksheets():
    print(f"  - {worksheet.title}")
print(f"\nTotal SKUs analyzed: {len(sku_details)}")
print(f"Channels: {', '.join(all_channels)}")
print(f"Forecast period: Feb 2026 - Dec 2026")
print(f"\nWhat's new in v3:")
print(f"  ✅ Seasonality by PRODUCT TYPE (24-month pooled indices, not per-item)")
print(f"  ✅ Seasonality table in Product Type Breakdown tab (month × product type)")
print(f"  ✅ Forecast sheets show 2024 + 2025 actuals alongside 2026 forecast")
print(f"  ✅ Actual vs Forecast columns color-coded (slate = actual, green = forecast)")
print(f"  ✅ Channel total row added to each forecast sheet")
print(f"  ✅ Aggregated By Year now includes 2026 Total Forecast column + TOTAL row")
print(f"  ✅ Inactivity rule: SKUs with 0 sales in last 6 months excluded from all forecasts")



DEMAND PLANNING TOOL v3 CREATED SUCCESSFULLY!

Google Sheet Name: Demand_Planning_20260214_0002
URL: https://docs.google.com/spreadsheets/d/1KTV8BnBjkd7yPYFnZ4LIEjIOGFoFk3Rj44tWbul2kpg

Sheets created:
  - Summary Dashboard
  - Product Type Breakdown
  - Direct-to-Consumer - Forecast
  - Wholesale - Forecast
  - TOTAL - Forecast
  - Aggregated By Year

Total SKUs analyzed: 275
Channels: Direct-to-Consumer, Wholesale, TOTAL
Forecast period: Feb 2026 - Dec 2026

What's new in v3:
  ✅ Seasonality by PRODUCT TYPE (24-month pooled indices, not per-item)
  ✅ Seasonality table in Product Type Breakdown tab (month × product type)
  ✅ Forecast sheets show 2024 + 2025 actuals alongside 2026 forecast
  ✅ Actual vs Forecast columns color-coded (slate = actual, green = forecast)
  ✅ Channel total row added to each forecast sheet
  ✅ Aggregated By Year now includes 2026 Total Forecast column + TOTAL row
  ✅ Inactivity rule: SKUs with 0 sales in last 6 months excluded from all forecasts
